# DITL with ADCS physics

This notebook simulates multiple slews driven by the ACS and plots reaction-wheel stored momentum over time.


In [ ]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from conops import MissionConfig, Queue, Pointing, QueueDITL
from conops.visualization import plot_ditl_telemetry
from rust_ephem import TLEEphemeris
import numpy as np
from tqdm import tqdm

# Load config and set inertia + wheel set
cfg = MissionConfig.from_json_file("../example_config.json")
cfg.spacecraft_bus.attitude_control.spacecraft_moi = (45.0, 45.0, 45.0)
cfg.spacecraft_bus.attitude_control.wheel_enabled = False  # use multi-wheel list
cfg.spacecraft_bus.attitude_control.wheels = [
    {"name": "rw1", "orientation": (+1,+1,+1), "max_torque": 0.06, "max_momentum": 1},
    {"name": "rw2", "orientation": (+1,-1,-1), "max_torque": 0.06, "max_momentum": 1},
    {"name": "rw3", "orientation": (-1,+1,-1), "max_torque": 0.06, "max_momentum": 1},
    {"name": "rw4", "orientation": (-1,-1,+1), "max_torque": 0.06, "max_momentum": 1},
]

# Configure 3 orthogonal magnetorquers (default B field is 3e-5 T; override if you want)
cfg.spacecraft_bus.attitude_control.magnetorquers = [
    {"name": "mtq_x", "orientation": (1, 0, 0), "dipole_strength": 32, "power_draw": 5.0},
    {"name": "mtq_y", "orientation": (0, 1, 0), "dipole_strength": 32, "power_draw": 5.0},
    {"name": "mtq_z", "orientation": (0, 0, 1), "dipole_strength": 32, "power_draw": 5.0},
]
cfg.spacecraft_bus.attitude_control.magnetorquer_bfield_T = 3e-5  # optional override

acs = cfg.spacecraft_bus.attitude_control
acs.cp_offset_body = (0.25, 0.0, 0.0)            # m
acs.residual_magnetic_moment = (0.05, 0, 0)      # A·m^2
acs.drag_area_m2 = 1.3 # m2
acs.drag_coeff = 2.2
acs.solar_area_m2 = 1.3
acs.solar_reflectivity = 1.3
cfg.spacecraft_bus.attitude_control.use_msis_density = True


begin = datetime(2025, 1, 1)
end = begin + timedelta(hours=6)
ephem = TLEEphemeris(tle="../example.tle", begin=begin, end=end, step_size=10)
cfg.constraint.ephem = ephem  # ensure ephem is set for PlanEntry/Pointing

# Build a busy queue
queue = Queue(config=cfg, ephem=ephem)
queue.slew_distance_weight = 40.0


number_of_targets = 1000
target_ra, target_dec = (
    np.random.uniform(0, 360, number_of_targets),
    np.random.uniform(-90, 90, number_of_targets),
)
print(f"Number of pointings = {len(target_ra)}")

targids = list(range(10000, 10000 + len(target_ra)))

targlist = Queue(
    ephem=ephem,
    config=cfg,
)
for i in tqdm(range(len(targids))):
    targlist.add(
        merit=40,
        ra=target_ra[i],
        dec=target_dec[i],
        obsid=targids[i],
        name=f"pointing_{targids[i]}",
        exptime=1000,
        ss_min=300,
    )


ditl = QueueDITL(config=cfg, ephem=ephem, begin=begin, end=end, queue=targlist)
ditl.step_size = 10  # seconds
ditl.calc()

fig, axes = plot_ditl_telemetry(ditl, figsize=(12, 10))
plt.show()



In [ ]:
# ruff: noqa: E402
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rust_ephem import TLEEphemeris
from tqdm import tqdm

from conops import MissionConfig, Queue, QueueDITL
from conops.common import ACSMode
from conops.visualization import plot_ditl_telemetry

# Shared ephemeris and time span
begin = datetime(2025, 1, 1)
end = begin + timedelta(hours=6)
ephem = TLEEphemeris(tle="../example.tle", begin=begin, end=end, step_size=10)

# Target set (fixed across sweeps)
number_of_targets = 1000
target_ra = np.random.uniform(0, 360, number_of_targets)
target_dec = np.random.uniform(-90, 90, number_of_targets)
targids = list(range(10000, 10000 + number_of_targets))

def make_base_config(mtq_dipole: float) -> MissionConfig:
    cfg_local = MissionConfig.from_json_file("../example_config.json")
    acs = cfg_local.spacecraft_bus.attitude_control
    acs.spacecraft_moi = (45.0, 45.0, 45.0)
    acs.wheel_enabled = False
    # Magnetorquers (dipole swept)
    acs.magnetorquers = [
        {"name": "mtq_x", "orientation": (1, 0, 0), "dipole_strength": mtq_dipole, "power_draw": 5.0},
        {"name": "mtq_y", "orientation": (0, 1, 0), "dipole_strength": mtq_dipole, "power_draw": 5.0},
        {"name": "mtq_z", "orientation": (0, 0, 1), "dipole_strength": mtq_dipole, "power_draw": 5.0},
    ]
    acs.magnetorquer_bfield_T = 3e-5
    acs.cp_offset_body = (0.25, 0.0, 0.0)
    acs.residual_magnetic_moment = (0.05, 0, 0)
    acs.drag_area_m2 = 1.3
    acs.drag_coeff = 2.2
    acs.solar_area_m2 = 1.3
    acs.solar_reflectivity = 1.3
    acs.use_msis_density = True
    cfg_local.constraint.ephem = ephem
    return cfg_local

def build_queue(cfg_local: MissionConfig) -> Queue:
    q = Queue(ephem=ephem, config=cfg_local)
    q.slew_distance_weight = 40.0
    for i in tqdm(range(number_of_targets), leave=False):
        q.add(
            merit=40,
            ra=target_ra[i],
            dec=target_dec[i],
            obsid=targids[i],
            name=f"pointing_{targids[i]}",
            exptime=1000,
            ss_min=300,
        )
    return q

# Wheel SKUs: (name, max_momentum [Nms], max_torque [Nm])
wheel_grid = [
    ("1.0Nms_0.06Nm", 1.0, 0.06),
    ("2.0Nms_0.12Nm", 2.0, 0.12),
    ("1.2Nms_0.25Nm", 1.2, 0.25),
    ("4.0Nms_0.25Nm", 4.0, 0.25),
    ("2.5Nms_0.50Nm", 2.5, 0.50),
    ("8.0Nms_0.25Nm", 8.0, 0.25),
    ("5.0Nms_0.50Nm", 5.0, 0.50),
]

# MTQ dipole sweep (A·m^2)
mtq_grid = [8, 16, 32, 50,100]

def run_case(name: str, max_mom: float, max_torque: float, mtq_dipole: float) -> tuple[QueueDITL, dict]:
    cfg_case = make_base_config(mtq_dipole)
    acs_case = cfg_case.spacecraft_bus.attitude_control
    acs_case.wheels = [
        {"name": f"rw{i}", "orientation": o, "max_torque": max_torque, "max_momentum": max_mom}
        for i, o in enumerate([(1,1,1), (1,-1,-1), (-1,1,-1), (-1,-1,1)], start=1)
    ]
    queue_case = build_queue(cfg_case)
    ditl_case = QueueDITL(config=cfg_case, ephem=ephem, begin=begin, end=end, queue=queue_case)
    ditl_case.step_size = 10
    ditl_case.calc()

    mode_arr = np.array(ditl_case.mode)
    science_frac = float(np.mean(mode_arr == ACSMode.SCIENCE) * 100)
    slew_frac = float(np.mean(mode_arr == ACSMode.SLEWING) * 100)
    desat_events = getattr(ditl_case.acs, "desat_events", 0)
    wm = np.array(getattr(ditl_case, "wheel_momentum_fraction", []))
    wt = np.array(getattr(ditl_case, "wheel_torque_fraction", []))

    summary = {
        "name": name,
        "mtq_dipole": mtq_dipole,
        "max_mom": max_mom,
        "max_torque": max_torque,
        "science_%": science_frac,
        "slew_%": slew_frac,
        "desat_events": desat_events,
        "max_mom_frac": float(wm.max()) if wm.size else 0.0,
        "p95_mom_frac": float(np.quantile(wm, 0.95)) if wm.size else 0.0,
        "max_torque_frac": float(wt.max()) if wt.size else 0.0,
    }
    return ditl_case, summary

results = []
ditls = {}
for mtq in mtq_grid:
    for name, mom, torq in wheel_grid:
        key = f"{name}_mtq{mtq}"
        ditl_case, summary = run_case(name, mom, torq, mtq)
        ditls[key] = ditl_case
        summary["key"] = key
        results.append(summary)

df = pd.DataFrame(results).set_index("key")
display(df)

# Example plot: choose a combination to visualize
case_to_plot = list(ditls.keys())[0]  # e.g., first combo
fig, axes = plot_ditl_telemetry(ditls[case_to_plot], figsize=(12, 8))
fig.suptitle(f"Case: {case_to_plot}")
plt.show()


In [ ]:
df_sorted = df.sort_values(
    by=["science_%", "slew_%", "desat_events", "max_mom_frac", "max_torque_frac"],
    ascending=[False, True, True, True, True],
)
display(df_sorted)


In [ ]:
for case_to_plot in ["1.0Nms_0.06Nm", "5.0Nms_0.50Nm", "8.0Nms_0.25Nm"]:
    fig, axes = plot_ditl_telemetry(ditls[case_to_plot], figsize=(12, 8))
    fig.suptitle(f"Wheel SKU: {case_to_plot}")
    plt.show()


In [ ]:
import numpy as np
from collections import Counter
from conops.common import ACSMode, ACSCommandType

# Timebase
t = np.array(ditl.utime)
dt = np.diff(t, prepend=t[0])
hrs = dt / 3600

m = np.array(ditl.mode)
time_by_mode = {
    "SCIENCE": np.sum(hrs[m == ACSMode.SCIENCE]),
    "SLEWING": np.sum(hrs[m == ACSMode.SLEWING]),
    "PASS": np.sum(hrs[m == ACSMode.PASS]),
    "SAFE": np.sum(hrs[m == ACSMode.SAFE]),
    "CHARGING": np.sum(hrs[m == ACSMode.CHARGING]),
    "SAA": np.sum(hrs[m == ACSMode.SAA]),
}
print("Hours by mode:", time_by_mode)

# Desat
desat_steps = getattr(ditl, "desat_time_steps", 0)
print(f"Desat hours: {desat_steps * ditl.step_size / 3600:.2f}")

# Wheel margins
wm = np.array(getattr(ditl, "wheel_momentum_fraction", []))
wt = np.array(getattr(ditl, "wheel_torque_fraction", []))
if wm.size:
    print(f"Wheel momentum: max={wm.max():.2f}, p95={np.quantile(wm,0.95):.2f}")
if wt.size:
    print(f"Wheel torque: max={wt.max():.2f}, p95={np.quantile(wt,0.95):.2f}")

# Slew outcomes
slews = [
    c for c in ditl.acs.executed_commands
    if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew
]
dists = [c.slew.slewdist for c in slews]
rejected = getattr(ditl.acs, "headroom_rejects", 0)
print(f"Slews: {len(slews)} executed, {rejected} rejected; dist mean={np.mean(dists):.1f} deg")

# Simple histograms (numpy bins)
hist_mom, bins_m = np.histogram(wm, bins=np.linspace(0,1,11)) if wm.size else ([],[])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

time_hours = (np.array(ditl.utime) - ditl.utime[0]) / 3600

plt.figure(figsize=(10,5))
#plt.plot(time_hours, ditl.disturbance_total, label="Total")
plt.plot(time_hours, ditl.disturbance_gg, label="Gravity gradient")
plt.plot(time_hours, ditl.disturbance_drag, label="Aero drag")
plt.plot(time_hours, ditl.disturbance_srp, label="Solar pressure")
plt.plot(time_hours, ditl.disturbance_mag, label="Magnetic (residual)")
plt.xlabel("Time (hours)")
plt.ylabel("Torque magnitude (N·m)")
plt.legend()
plt.yscale("log")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from datetime import datetime, timezone

# enable MSIS
cfg.spacecraft_bus.attitude_control.use_msis_density = True

# after your DITL calc:
acs = ditl.acs
ephem = ditl.ephem

densities = []
alts_km = []
for ut in ditl.utime:
    idx = ephem.index(datetime.fromtimestamp(ut, tz=timezone.utc))
    r_vec = np.array(ephem.gcrs_pv.position[idx])
    # normalize to meters if in km
    if np.linalg.norm(r_vec) < 1e6:
        r_vec = r_vec * 1000.0
    alt_m = max(0.0, np.linalg.norm(r_vec) - 6378e3)
    lat = float(getattr(ephem, "lat", [0])[idx]) if hasattr(ephem, "lat") else 0.0
    lon = float(getattr(ephem, "long", [0])[idx]) if hasattr(ephem, "long") else 0.0
    rho = acs._atmospheric_density(ut, alt_m, lat, lon)
    densities.append(rho)
    alts_km.append(alt_m / 1000.0)

# Plot density vs time
import matplotlib.pyplot as plt
time_hours = (np.array(ditl.utime) - ditl.utime[0]) / 3600
plt.semilogy(time_hours, densities)
plt.xlabel("Time (hours)")
plt.ylabel("Density (kg/m^3)")
plt.tight_layout()
plt.show()


In [ ]:
acs = ditl.acs
acs._use_msis = True  # ensure MSIS path
ut0 = ditl.utime[0]
idx = ditl.ephem.index(datetime.fromtimestamp(ut0, tz=timezone.utc))
r = np.array(ditl.ephem.gcrs_pv.position[idx])
if np.linalg.norm(r) < 1e6: r = r*1000
alt = np.linalg.norm(r) - 6378e3
lat = float(getattr(ditl.ephem, "lat", [0])[idx])
lon = float(getattr(ditl.ephem, "long", [0])[idx])
rho = acs._atmospheric_density(ut0, alt, lat, lon)
print(f"alt={alt/1000:.1f} km rho={rho:.3e} kg/m^3")


In [ ]:
alt

In [ ]:
ditl.print_statistics()

In [ ]:
from conops.common import ACSCommandType, ACSMode
time_h = (np.array(ditl.utime) - ditl.utime[0]) / 3600
slew_cmds = [c for c in ditl.acs.executed_commands if c.command_type == ACSCommandType.SLEW_TO_TARGET]
desat_cmds = [c for c in ditl.acs.executed_commands if c.command_type == ACSCommandType.DESAT]
print("DESAT windows:", [(c.execution_time, getattr(c, "duration", None)) for c in desat_cmds])

# Overlay where mode==SLEWING to see it spans settle/desat
import matplotlib.pyplot as plt
m = np.array(ditl.mode)
plt.plot(time_h, ditl.ra)
plt.scatter(time_h[m == ACSMode.SLEWING], np.array(ditl.ra)[m == ACSMode.SLEWING], s=8, color="r")


In [ ]:
from conops.common import ACSCommandType

cmds = [c for c in ditl.acs.executed_commands if c.command_type == ACSCommandType.SLEW_TO_TARGET]
for c in cmds:
    dist = c.slew.slewdist          # deg along great circle
    t = c.slew.slewtime             # s (includes settle)
    avg_rate = dist / t if t else 0
    a = getattr(c.slew, "_accel_override", None)
    vmax = getattr(c.slew, "_vmax_override", None)
    print(f"obsid={c.slew.obsid} dist={dist:.2f}deg time={t:.1f}s avg={avg_rate:.3f} deg/s accel={a} vmax={vmax}")


In [ ]:
import numpy as np
from conops.common import separation

# great-circle step-to-step rate
rates = []
for i in range(1, len(ditl.ra)):
    r = separation(
        [np.deg2rad(ditl.ra[i-1]), np.deg2rad(ditl.dec[i-1])],
        [np.deg2rad(ditl.ra[i]),   np.deg2rad(ditl.dec[i])]
    ) # deg
    d = np.rad2deg(r)
    rates.append(d / ditl.step_size)  # deg/s
print("Max step rate:", max(rates), "deg/s")


In [ ]:
import numpy as np
from conops.common import ACSCommandType
dists = [c.slew.slewdist for c in ditl.acs.executed_commands
         if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew]
print("mean dist:", np.mean(dists), "deg", "max:", np.max(dists), "deg")


In [ ]:
import numpy as np
from conops.common import ACSCommandType, separation

# Slew distances actually flown
dists = [c.slew.slewdist for c in ditl.acs.executed_commands
         if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew]
print(f"Count={len(dists)}, mean={np.mean(dists):.1f} deg, median={np.median(dists):.1f}, max={np.max(dists):.1f}")

# Time in SLEWING
mode_array = np.array(ditl.mode)
frac_slewing = np.mean(mode_array == 1) * 100
print(f"SLEWING fraction: {frac_slewing:.1f}%")

# Effective accel/rate per slew
for c in ditl.acs.executed_commands:
    if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew:
        print(f"obsid={c.slew.obsid} dist={c.slew.slewdist:.1f} deg, "
              f"accel={getattr(c.slew,'_accel_override',None)}, "
              f"vmax={getattr(c.slew,'_vmax_override',None)}")


In [ ]:
import numpy as np
from conops.common import ACSMode

mode_arr = np.array(ditl.mode)
science_frac = np.mean(mode_arr == ACSMode.SCIENCE) * 100
slew_frac = np.mean(mode_arr == ACSMode.SLEWING) * 100
print(f"SCIENCE: {science_frac:.1f}%")
print(f"SLEWING: {slew_frac:.1f}%")
